# 기본 과제 1번

---
## 연산
- 피연산자가 중 하나라도 tensor 타입이면 결과는 tensor로 나온다.

In [1]:
import torch

A = torch.Tensor([5])
B = torch.Tensor([7])

print(torch.add(A, B))
print(A + B)

tensor([12.])
tensor([12.])


---
## 인덱싱, 슬라이싱, 조인 등..
### ```torch.index_select()```
- torch.index_select(input, dim, index, *, out=None)
- index 범위를 따로 만들고 index_select 함수안에 파라미터로 넣는게 편함

In [6]:
A = torch.Tensor([[1, 2],
                  [3, 4]])

# TODO : [1, 3]을 만드세요!

# torch.index_select 함수를 써서 해보세요!
indices = torch.tensor([0])
output = torch.index_select(A, 1, indices)

print(output)
print('1차원 벡터로 변환:', output.view(-1))

tensor([[1.],
        [3.]])
1차원 벡터로 변환: tensor([1., 3.])


### ```torch.gather()```
- torch.gather(input, dim, index, *, sparse_grad=False, out=None)
- 2차원, 3차원... 등에 사용
- index는 input과 차원이 같아야한다!!

In [7]:
# dim=1 이니까 열 방향으로 indexing 진행
t = torch.tensor([[1, 2], [3, 4]])
torch.gather(t, 1, torch.tensor([[0, 0], [1, 0]]))

tensor([[1, 1],
        [4, 3]])

In [11]:
A = torch.Tensor([[1, 2],
                  [3, 4]])

# 1차원으로 줄이고 인덱싱도 가능
B = A.view(-1)
print(torch.gather(B, 0, torch.tensor([0,3])))
print('----')

# torch.gather 함수를 써서 해보세요!
# A에서 0(행방향)으로 인덱싱
output = torch.gather(A, 0, torch.tensor([[0,1], [0,1]]))
# print(output)
print(output[1])

tensor([1., 4.])
----
tensor([[1., 4.],
        [1., 4.]])
tensor([1., 4.])


In [15]:
### 고난도, 3D tensor gather -> 2차원

# TODO : 임의의 크기의 3D tensor에서 대각선 요소 가져와 2D로 반환하는 함수를 만드세요! 
def get_diag_element_3D(A):

    c = A.shape[0]
    h = A.shape[1]
    w = A.shape[2]

    # 행과 열 중 어디가 기느냐에 따라 대각요소를 뽑는 범위가 달라진다. (더 짧은 쪽에 맞춰서 뽑아야 함)
    if h <= w:
        ind = torch.arange(h).expand(c,h).reshape(c,1,h)
        output = torch.gather(A, 1, ind).view(-1,h)
    else:
        ind = torch.arange(w).expand(c,w).reshape(c,1,w)
        output = torch.gather(A, 1, ind).view(-1,w)

    return output

In [18]:
A = torch.Tensor([[[1, 2, 3],
                   [4, 5, 6]]])
print(get_diag_element_3D(A))

A = torch.tensor([[[ 1,  2,  3,  4,  5],
                   [ 6,  7,  8,  9, 10],
                   [11, 12, 13, 14, 15]],
          
                  [[16, 17, 18, 19, 20],
                   [21, 22, 23, 24, 25],
                   [26, 27, 28, 29, 30]]])
get_diag_element_3D(A)

tensor([[1., 5.]])


tensor([[ 1,  7, 13],
        [16, 22, 28]])

### ```torch.chunk()```
- torch.chunk(input, chunks, dim=0)
- tensor를 split 해주는 기능

In [23]:
import numpy as np

#  torch.chunk
t = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])

print(torch.chunk(t, 2, 0)) # 행 방향으로 2등분
print(torch.chunk(t, 2, 1)) # 열 방향으로 2등분
print(torch.chunk(t, 3, 1)) # 열 방향으로 3등분

(tensor([[1, 2, 3]]), tensor([[4, 5, 6]]))
(tensor([[1, 2],
        [4, 5]]), tensor([[3],
        [6]]))
(tensor([[1],
        [4]]), tensor([[2],
        [5]]), tensor([[3],
        [6]]))


### ```torch.swapdims()```
- torch.swapdims(input, dim0, dim1)
- torch.transpose()과 같은 기능이라 생각하면 된다.

In [23]:
x = torch.tensor([[[0,1],[2,3]],[[4,5],[6,7]]])
print(torch.swapdims(x, 0, 1))
print(torch.swapdims(x, 0, 2))

### ```torch.scatter()``` ???
- Tensor.scatter_(dim, index, src, reduce=None)
- 공식문서 : https://pytorch.org/docs/stable/generated/torch.Tensor.scatter_.html#torch.Tensor.scatter_

In [3]:
src = torch.arange(1, 11).reshape((2, 5))

index = torch.tensor([[0, 1, 2, 0]])
print(torch.zeros(3, 5, dtype=src.dtype).scatter_(0, index, src))

index = torch.tensor([[0, 1, 2], [0, 1, 4]])
print(torch.zeros(3, 5, dtype=src.dtype).scatter_(1, index, src))


tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0]])
tensor([[1, 2, 3, 0, 0],
        [6, 7, 0, 0, 8],
        [0, 0, 0, 0, 0]])


### ```torch.einsum()``` ???
- torch.einsum(equation, *operands)
- 공식문서: https://pytorch.org/docs/stable/generated/torch.einsum.html#torch.einsum

In [25]:
torch.einsum('ii', torch.randn(4, 4))

torch.einsum('ii->i', torch.randn(4, 4))

x = torch.randn(5)
y = torch.randn(4)
print(torch.einsum('i,j->ij', x, y))

As = torch.randn(3,2,5)
Bs = torch.randn(3,5,4)
print(torch.einsum('bij,bjk->bik', As, Bs))

tensor([[ 0.4996, -1.3301,  0.3208, -0.5237],
        [-0.2794,  0.7438, -0.1794,  0.2929],
        [-0.2183,  0.5810, -0.1401,  0.2288],
        [ 0.0606, -0.1612,  0.0389, -0.0635],
        [-0.4102,  1.0921, -0.2634,  0.4300]])
tensor([[[ 1.4559, -0.0908,  3.8621, -0.3544],
         [-1.8633,  0.9283, -2.3702, -1.8273]],

        [[ 2.4102, -4.2638, -6.2387,  6.2946],
         [ 0.6687, -2.3406, -2.4810,  2.6455]],

        [[ 2.5591,  2.6877, -4.0414, -3.2840],
         [-0.7194,  1.5803,  3.4003,  1.6086]]])


---
## Module 클래스
### ```torch.nn.Module```
- 모든 신경망 모듈의 기본 클래스입니다.
- 모듈은 트리 구조에 중첩할 수 있도록 다른 모듈을 포함할 수도 있습니다. 하위 모듈을 일반 특성으로 할당할 수 있습니다.
- 결국 기능들을 한 곳에 모아놓는 상자 역할을 한다.
- 내부에 여러가지 method들을 사용할 수 있다.
- ex) add_module, apply, buffers, children, hook, forward 등등

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__() ### init 과정에 반드시 super 코드가 들어감
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

### ```nn.Sequental()```
- 모듈을 하나로 묶어서 순차적으로 실행시킬 수 있다
- ```nn.ModuleList```은 말 그대로 모듈을 리스트에 저장만 함
- ```nn.ModuleDict```은 말 그대로 모듈을 딕셔너리에 저장만 함

In [9]:
# TODO : 다음의 모듈(Module)을 읽고 이해해보세요!
class Add(nn.Module):
    def __init__(self, value):
        super().__init__()
        self.value = value
#         print(self.value)

    def forward(self, x):
#         print(x, self.value)
        return x + self.value

# TODO : 위에 모듈(Module)과 nn.Sequential를 이용해서
#        입력값 x가 주어지면 다음의 연산을 처리하는 모델을 만들어보세요!
#        y = x + 3 + 2 + 5
calculator = nn.Sequential(Add(3), Add(2), Add(5))
print('합:', calculator(torch.tensor([1])))

합: tensor([11])


### ```딥러닝 모델 만들기```
- 최소 기능 단위인 Function을 만들고
- Function들로 이루어진 Layer을 만들고
- Layer들로 하나의 큰 Model을 만든다

In [11]:
import torch
from torch import nn


# Function
class Function_A(nn.Module):
    def __init__(self):
        super().__init__()
        print(f"        Function A Initialized")

    def forward(self, x):
        print(f"        Function A started")
        print(f"        Function A done")

class Function_B(nn.Module):
    def __init__(self):
        super().__init__()
        print(f"        Function B Initialized")

    def forward(self, x):
        print(f"        Function B started")
        print(f"        Function B done")

class Function_C(nn.Module):
    def __init__(self):
        super().__init__()
        print(f"        Function C Initialized")

    def forward(self, x):
        print(f"        Function C started")
        print(f"        Function C done")

class Function_D(nn.Module):
    def __init__(self):
        super().__init__()
        print(f"        Function D Initialized")

    def forward(self, x):
        print(f"        Function D started")
        print(f"        Function D done")


# Layer
class Layer_AB(nn.Module):
    def __init__(self):
        super().__init__()

        self.a = Function_A()
        self.b = Function_B()

        print(f"    Layer AB Initialized")

    def forward(self, x):
        print(f"    Layer AB started")
        self.a(x)
        self.b(x)
        print(f"    Layer AB done")

class Layer_CD(nn.Module):
    def __init__(self):
        super().__init__()

        self.c = Function_C()
        self.d = Function_D()

        print(f"    Layer CD Initialized")

    def forward(self, x):
        print(f"    Layer CD started")
        self.c(x)
        self.d(x)
        print(f"    Layer CD done")


# Model
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.ab = Layer_AB()
        self.cd = Layer_CD()

        print(f"Model ABCD Initialized\n")

    def forward(self, x):
        print(f"Model ABCD started")
        self.ab(x)
        self.cd(x)
        print(f"Model ABCD done\n")


x = torch.tensor([7])

model = Model()
model(x)

        Function A Initialized
        Function B Initialized
    Layer AB Initialized
        Function C Initialized
        Function D Initialized
    Layer CD Initialized
Model ABCD Initialized

Model ABCD started
    Layer AB started
        Function A started
        Function A done
        Function B started
        Function B done
    Layer AB done
    Layer CD started
        Function C started
        Function C done
        Function D started
        Function D done
    Layer CD done
Model ABCD done



### ```Parameter```
- 선형회귀 ```Y = XW + b```에서 W와 b가 파라미터가 된다.
- torch.nn.parameter.Parameter(data=None, requires_grad=True)
- Parameter로 만들어야 W와 b가 저장이 된다.
- ```state_dict()```를 통해 모델의 파라미터들을 확인할 수 있음
- 하지만 대부분 torch.nn에 구현된 레이어를 사용하기 때문에 직접 지정할 일은 없음

In [15]:
from torch.nn.parameter import Parameter

# 예시
# 1로 초기화하기 위해 torch.ones()를 사용했음
out_features, in_features = 10, 5
self.W = Parameter(torch.ones((out_features, in_features)))
self.b = Parameter(torch.ones((out_features)))

---
## 모델 분석하기
#### [위에서 만든 딥러닝 모델을 갖고 사용함](#딥러닝-모델-만들기)
- 모델에서 어떤 module이 있는지 확인하기

In [16]:
model

Model(
  (ab): Layer_AB(
    (a): Function_A()
    (b): Function_B()
  )
  (cd): Layer_CD(
    (c): Function_C()
    (d): Function_D()
  )
)

### ```named_children```
- 어떤 모듈(module)들이 있는지 보여줌
- 대신, 한 단계 아래의 submodule까지만 보여줌

In [18]:
for name, child in model.named_children():
    print(f"[ Name ] : {name}\n[ Children ]\n{child}")
    print("-" * 30)

[ Name ] : ab
[ Children ]
Layer_AB(
  (a): Function_A()
  (b): Function_B()
)
------------------------------
[ Name ] : cd
[ Children ]
Layer_CD(
  (c): Function_C()
  (d): Function_D()
)
------------------------------


### ```named_modules```
- 어떤 모듈(module)들이 있는지 보여줌
- ```named_children```과 달리 자신에게 속하는 모든 submodule을 보여줌
- 예를 들어, self.a = Function_A() 라면 Function_A 모듈의 name이 'a' 가 된다

In [19]:
for name, module in model.named_modules():
    print(f"[ Name ] : {name}\n[ Module ]\n{module}")
    print("-" * 30)

[ Name ] : 
[ Module ]
Model(
  (ab): Layer_AB(
    (a): Function_A()
    (b): Function_B()
  )
  (cd): Layer_CD(
    (c): Function_C()
    (d): Function_D()
  )
)
------------------------------
[ Name ] : ab
[ Module ]
Layer_AB(
  (a): Function_A()
  (b): Function_B()
)
------------------------------
[ Name ] : ab.a
[ Module ]
Function_A()
------------------------------
[ Name ] : ab.b
[ Module ]
Function_B()
------------------------------
[ Name ] : cd
[ Module ]
Layer_CD(
  (c): Function_C()
  (d): Function_D()
)
------------------------------
[ Name ] : cd.c
[ Module ]
Function_C()
------------------------------
[ Name ] : cd.d
[ Module ]
Function_D()
------------------------------


### ```get_submodule```
- 모델 안에 있는 모듈 중, 특정 모듈만 가져오고 싶음
- 모듈의 name을 사용해서 특정 모듈을 가져올 수 있음

In [28]:
# Function_A 모듈을 가져오기
# a를 가져와야하는데 a는 ab에 속한다.
submodule = model.get_submodule('ab.a')
print(submodule)
print(submodule.__class__)

### ```named_parameters```
- 각 모듈들의 파라미터를 반환해준다
- weight와 bias 의 값을 알 수 있음

In [37]:
for i in model.named_parameters():
    print(i)

# ㅇ
for name, parameter in model.named_parameters():
    print(f"[ Name ] : {name}\n[ Parameter ]\n{parameter}")
    print("-" * 30)

### ```get_parameter```
- ```get_submodule```과 동일하게 사용한다. (모듈의 name을 사용함)
- 타겟의 파라미터 값을 반환한다.

In [39]:
# ab 안의 b의 Weight1 값을 가져옴
parameter = model.get_parameter('ab.b.W1')
parameter